In [1]:
!pip install yfinance gspread oauth2client scikit-learn python-telegram-bot --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.2 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.2 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.2 which is incompatible.


In [2]:
!pip install --upgrade yfinance pandas_ta gspread oauth2client --quiet


In [3]:
!pip install yfinance gspread oauth2client --quiet


In [2]:

!pip install numpy==1.25.2 scipy==1.11.4 scikit-learn==1.3.0 pandas-ta==0.3.14b --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.1 MB/s eta 0:00:00
  Using cached pandas_ta-0.3.14b0-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 23.5 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests


In [2]:
def fetch_stock_data(ticker, period="6mo", interval="1d"):
    df = yf.download(ticker, period=period, interval=interval)
    df.dropna(inplace=True)
    return df

tickers = ['TCS.NS', 'INFY.NS', 'RELIANCE.NS']
stock_data = {ticker: fetch_stock_data(ticker) for ticker in tickers}


/tmp/ipython-input-1433583922.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1433583922.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1433583922.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed


In [3]:
def compute_rsi(df, period=14):
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi



In [4]:
def apply_indicators(df):
    df['RSI'] = compute_rsi(df)
    df['20DMA'] = df['Close'].rolling(window=20).mean()
    df['50DMA'] = df['Close'].rolling(window=50).mean()
    df.dropna(inplace=True)
    return df

for ticker in tickers:
    stock_data[ticker] = apply_indicators(stock_data[ticker])


In [5]:
def generate_signals(df):
    signals = []
    for i in range(1, len(df)):
        if (
            df['RSI'].iloc[i] < 30
            and df['20DMA'].iloc[i] > df['50DMA'].iloc[i]
            and df['20DMA'].iloc[i - 1] <= df['50DMA'].iloc[i - 1]
        ):
            signals.append(("BUY", df.index[i], df['Close'].iloc[i]))
        elif df['RSI'].iloc[i] > 70:
            signals.append(("SELL", df.index[i], df['Close'].iloc[i]))
    return signals

signals = {ticker: generate_signals(stock_data[ticker]) for ticker in tickers}


In [6]:
def backtest_signals(df, signals):
    trades = []
    position = None

    for i in range(len(signals)):
        action, date, price = signals[i]

        if action == "BUY" and position is None:
            position = (date, price)  # Open position
        elif action == "SELL" and position is not None:
            buy_date, buy_price = position
            sell_date, sell_price = date, price
            profit = sell_price - buy_price
            trades.append({
                "Buy Date": buy_date,
                "Buy Price": buy_price,
                "Sell Date": sell_date,
                "Sell Price": sell_price,
                "P&L": profit
            })
            position = None

    # Close open position at the end
    if position is not None:
        buy_date, buy_price = position
        sell_price = df['Close'].iloc[-1]
        profit = sell_price - buy_price
        trades.append({
            "Buy Date": buy_date,
            "Buy Price": buy_price,
            "Sell Date": df.index[-1],
            "Sell Price": sell_price,
            "P&L": profit
        })

    return pd.DataFrame(trades)


In [7]:
trade_logs = {}
for ticker in tickers:
    df = stock_data[ticker]
    trade_logs[ticker] = backtest_signals(df, signals[ticker])


In [8]:
for ticker in tickers:
    print(f"\n📈 {ticker} Trade Summary:")
    log = trade_logs[ticker]
    if not log.empty:
        total_trades = len(log)
        win_trades = sum(log["P&L"] > 0)
        win_ratio = win_trades / total_trades * 100
        total_profit = log["P&L"].sum()
        print(f"Total Trades: {total_trades}")
        print(f"Winning Trades: {win_trades}")
        print(f"Win Ratio: {win_ratio:.2f}%")
        print(f"Total P&L: ₹{total_profit:.2f}")
    else:
        print("No trades generated.")



📈 TCS.NS Trade Summary:
No trades generated.

📈 INFY.NS Trade Summary:
No trades generated.

📈 RELIANCE.NS Trade Summary:
No trades generated.


In [9]:
from google.colab import files
uploaded = files.upload()



Saving Credentials.json to Credentials (1).json


In [10]:
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

# Define scope for Google Sheets access
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Load credentials from uploaded JSON
creds = Credentials.from_service_account_file("Credentials.json", scopes=scope)
gc = gspread.authorize(creds)


In [11]:
sh = gc.open("AlgoTradingSheet")
tcs_data_ws = sh.worksheet("TCS")           # Main stock data
tcs_trades_ws = sh.worksheet("TCS_Trades")  # Trade signals/log



In [12]:
data = tcs_trades_ws.get_all_records()
print(data)


[{'Date': '2025-08-06', 'Signal': 'BUY', 'Price': 3615, 'RSI': 29, '20DMA': 3580, '50DMA': 3550}, {'Date': '2025-08-07', 'Signal': 'SELL', 'Price': 3700, 'RSI': 70, '20DMA': 3650, '50DMA': 3600}]


In [13]:
# Example DataFrame
import pandas as pd

df_trades = pd.DataFrame([
    {"Date": "2025-08-06", "Signal": "BUY", "Price": 3615, "RSI": 29, "20DMA": 3580, "50DMA": 3550},
    {"Date": "2025-08-07", "Signal": "SELL", "Price": 3700, "RSI": 70, "20DMA": 3650, "50DMA": 3600}
])


In [14]:
from gspread_dataframe import set_with_dataframe

set_with_dataframe(tcs_trades_ws, df_trades)


In [15]:
import gspread
from datetime import datetime
from google.oauth2.service_account import Credentials

# Assuming you've already authenticated and have the sheet
# Example: sh = gc.open_by_key("your_spreadsheet_key")
tcs_trades_ws = sh.worksheet("TCS_Trades")

def log_trade_to_sheet(date, action, price, rsi, dma_20, dma_50, result="Open"):
    row = [date, action, price, rsi, dma_20, dma_50, result]
    tcs_trades_ws.append_row(row)


In [16]:
print(stock_data.keys())


dict_keys(['TCS.NS', 'INFY.NS', 'RELIANCE.NS'])


In [17]:
latest = stock_data["TCS.NS"].iloc[-1]


In [18]:
rsi = latest["RSI"]
dma_20 = latest["20DMA"]
dma_50 = latest["50DMA"]
price = latest["Close"]





In [19]:
# Use the correct key — print(stock_data.keys()) to confirm if needed
latest = stock_data["TCS.NS"].iloc[-1]  # or "TCS" if your dictionary uses that key

# Force single float values from the row
rsi = float(latest["RSI"])
dma_20 = float(latest["20DMA"])
dma_50 = float(latest["50DMA"])
price = float(latest["Close"])

# Apply the trading logic
if (rsi < 30) and (dma_20 > dma_50):
    action = "Buy"
elif (rsi > 70) and (dma_20 < dma_50):
    action = "Sell"
else:
    action = None

# Log trade to sheet if action triggered
if action:
    log_trade_to_sheet(
        date=datetime.today().strftime("%Y-%m-%d"),
        action=action,
        price=price,
        rsi=rsi,
        dma_20=dma_20,
        dma_50=dma_50
    )
    print(f"{action} trade logged.")
else:
    print("No trade signal today.")


No trade signal today.


/tmp/ipython-input-2401558020.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  rsi = float(latest["RSI"])
/tmp/ipython-input-2401558020.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dma_20 = float(latest["20DMA"])
/tmp/ipython-input-2401558020.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dma_50 = float(latest["50DMA"])
/tmp/ipython-input-2401558020.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(latest["Close"])


In [20]:
import yfinance as yf
import pandas as pd
import numpy as np



In [21]:
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


In [22]:
ticker = "TCS.NS"
data = yf.download(ticker, period="3mo")

data["20DMA"] = data["Close"].rolling(window=20).mean()
data["50DMA"] = data["Close"].rolling(window=50).mean()
data["RSI"] = compute_rsi(data["Close"], period=14)

data.dropna(inplace=True)

# Store in dictionary like before
stock_data = {ticker: data}


/tmp/ipython-input-2276325484.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="3mo")
[*********************100%***********************]  1 of 1 completed


In [23]:
def log_trade_to_sheet(date, action, price, rsi, dma_20, dma_50, result="Open"):
    row = [date, action, price, rsi, dma_20, dma_50, result]
    tcs_trades_ws.append_row(row)


In [24]:
print(stock_data.keys())


dict_keys(['TCS.NS'])


In [25]:
# Ensure you're extracting the latest row correctly
latest = stock_data["TCS.NS"].iloc[-1]  # This MUST return a Series, not DataFrame

# Convert each to scalar value using `.item()` if needed
rsi = latest["RSI"].item() if hasattr(latest["RSI"], "item") else latest["RSI"]
dma_20 = latest["20DMA"].item() if hasattr(latest["20DMA"], "item") else latest["20DMA"]
dma_50 = latest["50DMA"].item() if hasattr(latest["50DMA"], "item") else latest["50DMA"]

# Now safe to compare
if (rsi < 30) and (dma_20 > dma_50):
    action = "Buy"
elif (rsi > 70) and (dma_20 < dma_50):
    action = "Sell"
else:
    action = "Hold"

print("Trade Action:", action)


Trade Action: Hold


In [26]:
from datetime import datetime

ticker = "TCS.NS"

# Get the latest row of TCS.NS stock data
latest = stock_data[ticker].iloc[-1]

# Extract indicator values as scalars
rsi = float(latest["RSI"])
dma_20 = float(latest["20DMA"])
dma_50 = float(latest["50DMA"])
price = float(latest["Close"])

# Apply trading strategy
if (rsi < 30) and (dma_20 > dma_50):
    action = "Buy"
elif (rsi > 70) and (dma_20 < dma_50):
    action = "Sell"
else:
    action = "Hold"

# Log only if Buy/Sell
if action != "Hold":
    log_trade_to_sheet(
        date=datetime.today().strftime("%Y-%m-%d"),
        action=action,
        price=price,
        rsi=rsi,
        dma_20=dma_20,
        dma_50=dma_50
    )

# Optional status output
print(f"{ticker} - Action: {action}, RSI: {rsi:.2f}, DMA20: {dma_20:.2f}, DMA50: {dma_50:.2f}")


TCS.NS - Action: Hold, RSI: 29.98, DMA20: 3128.36, DMA50: 3299.47


/tmp/ipython-input-294717985.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  rsi = float(latest["RSI"])
/tmp/ipython-input-294717985.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dma_20 = float(latest["20DMA"])
/tmp/ipython-input-294717985.py:11: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dma_50 = float(latest["50DMA"])
/tmp/ipython-input-294717985.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(latest["Close"])


In [27]:
from datetime import datetime
import yfinance as yf
import pandas as pd
import pandas_ta as ta

# List of tickers to track
tickers = ["TCS.NS", "INFY.NS", "RELIANCE.NS"]  # Add more as needed

# Dictionary to store stock data
stock_data = {}

# Fetch and calculate indicators for each stock
for ticker in tickers:
    data = yf.download(ticker, period="3mo")
    data["RSI"] = ta.rsi(data["Close"], length=14)
    data["20DMA"] = data["Close"].rolling(window=20).mean()
    data["50DMA"] = data["Close"].rolling(window=50).mean()
    stock_data[ticker] = data

# Apply trading strategy
for ticker in tickers:
    latest = stock_data[ticker].iloc[-1]

    try:
        rsi = float(latest["RSI"])
        dma_20 = float(latest["20DMA"])
        dma_50 = float(latest["50DMA"])
        price = float(latest["Close"])
    except Exception as e:
        print(f"Skipping {ticker} due to data issue: {e}")
        continue

    # Apply strategy
    if (rsi < 30) and (dma_20 > dma_50):
        action = "Buy"
    elif (rsi > 70) and (dma_20 < dma_50):
        action = "Sell"
    else:
        action = "Hold"

    # Log only Buy/Sell
    if action != "Hold":
        log_trade_to_sheet(
            date=datetime.today().strftime("%Y-%m-%d"),
            action=action,
            price=price,
            rsi=rsi,
            dma_20=dma_20,
            dma_50=dma_50
        )

    print(f"{ticker} - Action: {action}, RSI: {rsi:.2f}, DMA20: {dma_20:.2f}, DMA50: {dma_50:.2f}")


/tmp/ipython-input-2919372134.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="3mo")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2919372134.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="3mo")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2919372134.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="3mo")
[*********************100%***********************]  1 of 1 completed

Skipping TCS.NS due to data issue: float() argument must be a string or a real number, not 'NoneType'
Skipping INFY.NS due to data issue: float() argument must be a string or a real number, not 'NoneType'
Skipping RELIANCE.NS due to data issue: float() argument must be a string or a real number, not 'NoneType'



/tmp/ipython-input-2919372134.py:25: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  rsi = float(latest["RSI"])


In [28]:
!pip install schedule


In [29]:
import schedule
import time

# Define your main trading job
def run_algo():
    print("Running automated trading script...")
    # Your full strategy code goes here
    # This is where you fetch data, apply logic, and log to Google Sheets
    # For example:
    from datetime import datetime

    tickers = ["TCS.NS", "INFY.NS", "RELIANCE.NS"]
    stock_data = {}

    for ticker in tickers:
        data = yf.download(ticker, period="3mo")
        data["RSI"] = ta.rsi(data["Close"], length=14)
        data["20DMA"] = data["Close"].rolling(window=20).mean()
        data["50DMA"] = data["Close"].rolling(window=50).mean()
        stock_data[ticker] = data

    for ticker in tickers:
        latest = stock_data[ticker].iloc[-1]

        try:
            rsi = float(latest["RSI"])
            dma_20 = float(latest["20DMA"])
            dma_50 = float(latest["50DMA"])
            price = float(latest["Close"])
        except Exception as e:
            print(f"Skipping {ticker} due to data issue: {e}")
            continue

        if (rsi < 30) and (dma_20 > dma_50):
            action = "Buy"
        elif (rsi > 70) and (dma_20 < dma_50):
            action = "Sell"
        else:
            action = "Hold"

        if action != "Hold":
            log_trade_to_sheet(
                date=datetime.today().strftime("%Y-%m-%d"),
                action=action,
                price=price,
                rsi=rsi,
                dma_20=dma_20,
                dma_50=dma_50
            )

        print(f"{ticker} - Action: {action}, RSI: {rsi:.2f}, DMA20: {dma_20:.2f}, DMA50: {dma_50:.2f}")

# Schedule the job at 9:15 AM (IST) daily
schedule.every().day.at("09:15").do(run_algo)

# Loop to keep checking
while True:
    schedule.run_pending()
    time.sleep(60)  # wait 1 minute


KeyboardInterrupt: 

In [11]:
print("Columns in df:", df.columns.tolist())


Columns in df: [('Close', 'TCS.NS'), ('High', 'TCS.NS'), ('Low', 'TCS.NS'), ('Open', 'TCS.NS'), ('Volume', 'TCS.NS'), ('RSI', ''), ('Target', '')]


In [17]:
# List all Google Sheets the service account has access to
files = gc.list_spreadsheet_files()
for f in files:
    print(f['name'])


AlgoTradingSheet
